Imports

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from datetime import datetime as dt

Declarations

In [143]:
MAIN_SITE = "https://www.hltv.org"
startPos=0
endPos=5700
offset=100
dates = []
team1 = []
team2 = []
win_team = []
results_loser = []
results_winner = []
event = []
type_of_game = []
link_to_game = []


In [144]:
#For loop for each file scraped 
for file_number in range(startPos,endPos+offset,offset):
    #for each file make BeautifulSoup
    with open(f"HTML/{file_number}.html", "r") as reader:
        html_string = reader.read()
        soup = BeautifulSoup(html_string, 'html.parser')
    # For each day in the file
    for day in soup.find_all(class_="results-sublist"):
        # Get all matches links

        allLinks = day.find_all('a')        
        for link in allLinks:
            if link.has_key('href'):
                href = link['href']
                if r'/matches/' in href:
                    link_to_game.append(MAIN_SITE+href)
        #set variable date for this whole box
        date=day.find_all("div",class_="standard-headline")[0].text
        #For each game in the day
        for game in day.find_all(class_="result-con"): # Each Match
            dates.append(date) # Date
            team1.append(game.find_all(class_="team")[0].text) # Team 1
            if game.find_all(class_="score-won"): #If winner exists
                results_winner.append(game.find_all(class_="score-won")[0].text) # Result Winning Team
                results_loser.append(game.find_all(class_="score-lost")[0].text) # Result Losing Team
            else: #If winner doesn't exist append tie score instead
                results_winner.append(game.find_all(class_="score-tie")[0].text) # Result Winning Team
                results_loser.append(game.find_all(class_="score-tie")[0].text) # Result Losing Team
            team2.append(game.find_all(class_="team")[0+1].text) # Team 2
            if game.find_all(class_="team team-won"):#If winner exists
                win_team.append(game.find_all(class_="team team-won")[0].text) # Winning Team
            else: #If winner doesn't exist append "Tie" instead
                win_team.append("Tie")
            event.append(game.find_all(class_="event")[0].text) # Event
            type_of_game.append(game.find_all(class_="star-cell")[0].text.replace("\n","")) # Type of Game

In [ ]:
driver_path = "C:\\Users\\Djoze\\Desktop\\chromedriver.exe"
wd = webdriver.Chrome(executable_path=driver_path)
for i, game_link in enumerate(link_to_game):
    file_name = f"HTMLGAMES/{i}-{dates[i]}-{team1[i]}-{team2[i]}.html"
    wd.get(link_to_game[i])
    time.sleep(5)
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(wd.page_source)

C:\Users\Djoze\AppData\Local\Temp\ipykernel_15600\2199323510.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path)


KeyboardInterrupt: 

In [37]:
dates_final = []
team1_final = []
team2_final= []
win_team_final = []
results_loser_final = []
results_winner_final = []
event_final = []
type_of_game_final = []
link_to_game_final = []
maps = []
overtime = []
team1_world_rank = []
team1_rounds = []
team1_rounds_ct = []
team1_rounds_t = []
team1_rating = []
team2_world_rank = []
team2_rounds = []
team2_rounds_ct = []
team2_rounds_t = []
team2_rating = []
ranks = []
scores = []
results_ct = []
results_t = []
match_id = []
path_to_save = "HTMLGAMES\DONE"

In [41]:
last_id = 5800

for j in range(last_id):

    file_of_game = f"{last_id-j}-{dates[j]}-{team1[j]}-{team2[j]}.html"
    number_of_maps = 0
    ranks = []
    scores = []
    results_ct = []
    results_t = []

    with open(f"HTMLGAMES/{file_of_game}", "r", encoding="utf-8") as reader:
        html_string = reader.read()
        soup = BeautifulSoup(html_string, 'html.parser')

  
    for rank in soup.find_all(class_= "teamRanking"):
        ranks.append(rank.text.split("#")[-1])

    for flex in soup.find_all(class_="mapholder"):
        for map_played in flex.find_all(class_="mapname"):
            if map_played.text != "TBA":
                match_id.append(f"{last_id-j}")
                maps.append(map_played.text)
                dates_final.append(dates[j])
                team1_final.append(team1[j])
                team2_final.append(team2[j])
                win_team_final.append(win_team[j])
                results_loser_final.append(results_loser[j])
                results_winner_final.append(results_winner[j])
                event_final.append(event[j])
                type_of_game_final.append(type_of_game[j])
                link_to_game_final.append(link_to_game[j])
                number_of_maps+=1
                team1_world_rank.append(ranks[0])
                team2_world_rank.append(ranks[1])

    for results in soup.find_all(class_="results-center"):
        if results.find(class_="results-center-half-score"):
            if results.find(class_="results-center-half-score").text:
                for result_ct in results.find_all(class_="ct"):
                    results_ct.append(result_ct.text)
                for result_t in results.find_all(class_="t"):
                    results_t.append(result_t.text)
            else:
                results_ct.append("-")
                results_t.append("-")
                results_ct.append("-")
                results_t.append("-")
        else:
            results_ct.append("-")
            results_t.append("-")
            results_ct.append("-")
            results_t.append("-")

    for map_exists in soup.find_all(class_="mapholder"):
        if map_exists.find_all(class_="results-team-score"):
                for score in map_exists.find_all(class_="results-team-score"):
                    scores.append(score.text)
        else:
            scores.append("-")
            scores.append("-")

    for i in range(0,number_of_maps*2,2):
        team1_rounds.append(scores[i])
        team2_rounds.append(scores[i+1])
        
        if  scores[i] == "-" or scores[i+1] == "-" or ((scores[i] == "0" and scores[i+1] == "1") or (scores[i] == "1" and scores[i+1] == "0")):
            team1_rounds_ct.append("-")
            team1_rounds_t.append("-")
            team2_rounds_ct.append("-")
            team2_rounds_t.append("-")
        else:
            team1_rounds_ct.append(results_ct[i])
            team2_rounds_ct.append(results_ct[i+1])
            team1_rounds_t.append(results_t[i+1])
            team2_rounds_t.append(results_t[i])
       
        if  scores[i] == "-" or scores[i+1] == "-":
                overtime.append("Not Played")
        else:
            if int(scores[i])+int(scores[i+1]) > 30:
                overtime.append("Yes")
            elif int(scores[i])+int(scores[i+1]) == 1:
                overtime.append("Forfeit")
            else:
                overtime.append("No")

    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)

    os.rename(f"HTMLGAMES/{file_of_game}",f"HTMLGAMES/DONE/{file_of_game}")

In [152]:
team1_world_rank = [rank.replace("\nUnranked\n", "0") for rank in team1_world_rank]
team2_world_rank = [rank.replace("\nUnranked\n", "0") for rank in team2_world_rank]
team1_rounds = [team.replace("-", "0") for team in team1_rounds]
team2_rounds = [team.replace("-", "0") for team in team2_rounds]
team1_rounds_ct = [team.replace("-", "0") for team in team1_rounds_ct]
team2_rounds_ct = [team.replace("-", "0") for team in team2_rounds_ct]
team1_rounds_t = [team.replace("-", "0") for team in team1_rounds_t]
team2_rounds_t = [team.replace("-", "0") for team in team2_rounds_t]
for i, date in enumerate(dates_final):
    dates_temp = dates_final[i].split(" ")
    dates_temp
    dates_final[i] = f"{dates_temp[3]} {dates_temp[1]} {dates_temp[2]} {dates_temp[4]}"
dates_final = [date.replace("st for", "") for date in dates_final]
dates_final = [date.replace("nd for", "") for date in dates_final]
dates_final = [date.replace("rd for", "") for date in dates_final]
dates_final = [date.replace("th for", "") for date in dates_final]


['18 October 2022',
 '18 October 2022',
 '18 October 2022',
 '18 October 2022',
 '18 October 2022',
 '18 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',
 '17 October 2022',


In [153]:
results_data_matches = pd.DataFrame(data={"ID":match_id,	"Date":dates_final,	"Team 1":team1_final,"Team 2":team2_final,	"Winner":win_team_final,	"Winner's Match Score":results_winner_final,	"Loser's Match Score":results_loser_final,	"Event":event_final,	"Best of":type_of_game_final,	"Link":link_to_game_final,	"Map":maps,	"Overtime":overtime,	"Team 1 Rank":team1_world_rank,	"Team 2 Rank":team2_world_rank,	"Team 1 Map Score":team1_rounds,	"Team 2 Map Score":team2_rounds,	"Team 1 CT Rounds":team1_rounds_ct,	"Team 2 CT Rounds":team2_rounds_ct,	"Team 1 T Rounds":team1_rounds_t,	"Team 2 T Rounds":team2_rounds_t})
results_data_matches

,ID,Date,Team 1,Team 2,Winner,Winner's Match Score,Loser's Match Score,Event,Best of,Link,Map,Overtime,Team 1 Rank,Team 2 Rank,Team 1 Map Score,Team 2 Map Score,Team 1 CT Rounds,Team 2 CT Rounds,Team 1 T Rounds,Team 2 T Rounds
0,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Inferno,No,206,226,16,4,11,0,5,4
1,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Overpass,No,206,226,16,11,6,7,5,9
2,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Dust2,Not Played,206,226,0,0,0,0,0,0
3,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Mirage,No,104,131,6,16,3,4,3,12
4,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Nuke,Yes,104,131,22,20,10,10,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Nuke,No,84,127,16,14,8,9,6,7
14162,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Overpass,No,84,127,16,9,8,2,8,7
14163,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Inferno,No,39,22,16,12,5,6,7,10
14164,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Ancient,No,39,22,4,16,4,5,0,11


In [154]:
results_data_matches_mod = results_data_matches.copy()
results_data_matches_mod

,ID,Date,Team 1,Team 2,Winner,Winner's Match Score,Loser's Match Score,Event,Best of,Link,Map,Overtime,Team 1 Rank,Team 2 Rank,Team 1 Map Score,Team 2 Map Score,Team 1 CT Rounds,Team 2 CT Rounds,Team 1 T Rounds,Team 2 T Rounds
0,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Inferno,No,206,226,16,4,11,0,5,4
1,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Overpass,No,206,226,16,11,6,7,5,9
2,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Dust2,Not Played,206,226,0,0,0,0,0,0
3,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Mirage,No,104,131,6,16,3,4,3,12
4,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Nuke,Yes,104,131,22,20,10,10,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Nuke,No,84,127,16,14,8,9,6,7
14162,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Overpass,No,84,127,16,9,8,2,8,7
14163,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Inferno,No,39,22,16,12,5,6,7,10
14164,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Ancient,No,39,22,4,16,4,5,0,11


In [155]:
columns_to_int = ["ID","Winner's Match Score","Loser's Match Score","Team 1 Rank","Team 2 Rank","Team 1 Map Score","Team 2 Map Score","Team 1 CT Rounds","Team 2 CT Rounds","Team 1 T Rounds","Team 2 T Rounds"]
for i in columns_to_int:
    results_data_matches_mod[f"{i}"] = results_data_matches_mod[f"{i}"].astype(int,errors="raise")

In [156]:
results_data_matches_filtered = results_data_matches_mod.copy()
results_data_matches_filtered

,ID,Date,Team 1,Team 2,Winner,Winner's Match Score,Loser's Match Score,Event,Best of,Link,Map,Overtime,Team 1 Rank,Team 2 Rank,Team 1 Map Score,Team 2 Map Score,Team 1 CT Rounds,Team 2 CT Rounds,Team 1 T Rounds,Team 2 T Rounds
0,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Inferno,No,206,226,16,4,11,0,5,4
1,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Overpass,No,206,226,16,11,6,7,5,9
2,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Dust2,Not Played,206,226,0,0,0,0,0,0
3,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Mirage,No,104,131,6,16,3,4,3,12
4,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Nuke,Yes,104,131,22,20,10,10,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Nuke,No,84,127,16,14,8,9,6,7
14162,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Overpass,No,84,127,16,9,8,2,8,7
14163,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Inferno,No,39,22,16,12,5,6,7,10
14164,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Ancient,No,39,22,4,16,4,5,0,11


In [157]:
rows_to_drop = results_data_matches_filtered[(results_data_matches_filtered["Overtime"] == "Not Played") | (results_data_matches_filtered["Overtime"] == "Forfeit")].index
results_data_matches_filtered.drop(rows_to_drop, inplace=True)

In [158]:
results_data_matches_filtered

,ID,Date,Team 1,Team 2,Winner,Winner's Match Score,Loser's Match Score,Event,Best of,Link,Map,Overtime,Team 1 Rank,Team 2 Rank,Team 1 Map Score,Team 2 Map Score,Team 1 CT Rounds,Team 2 CT Rounds,Team 1 T Rounds,Team 2 T Rounds
0,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Inferno,No,206,226,16,4,11,0,5,4
1,5800,18 October 2022,Antic,Warriors,Antic,2,0,ESL Australia & NZ Championship Season 15,bo3,https://www.hltv.org/matches/2359361/antic-vs-...,Overpass,No,206,226,16,11,6,7,5,9
3,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Mirage,No,104,131,6,16,3,4,3,12
4,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Nuke,Yes,104,131,22,20,10,10,5,5
5,5799,18 October 2022,Victorum,Villainous,Victorum,2,1,ESL Challenger League Season 42 North America ...,bo3,https://www.hltv.org/matches/2359683/victorum-...,Ancient,Yes,104,131,19,17,11,11,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Nuke,No,84,127,16,14,8,9,6,7
14162,2,2 October 2022,ECLOT,Iron Branch,Iron Branch,2,1,ESEA Advanced Season 42 Europe,bo3,https://www.hltv.org/matches/2359320/eclot-vs-...,Overpass,No,84,127,16,9,8,2,8,7
14163,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Inferno,No,39,22,16,12,5,6,7,10
14164,1,2 October 2022,HOTU,TAG,TAG,2,1,FASTCUP Grand Tour 2022,bo3,https://www.hltv.org/matches/2359364/hotu-vs-t...,Ancient,No,39,22,4,16,4,5,0,11


In [159]:
results_data_matches_filtered.to_excel("Output.xlsx", index = False)